# MultiHop Planner Agent

### Load the Data and create Tools

In [ ]:
# !mkdir -p 'data/10k/'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(
    model="gpt-4o",
    temperature=0.1,
)
Settings.embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document
from llama_index.core.tools import QueryEngineTool

index = VectorStoreIndex.from_documents([Document.example()])
tool = QueryEngineTool.from_defaults(
    index.as_query_engine(),
    name="dummy",
    description="dummy",
)
# # Load documents, create tools
# lyft_documents = SimpleDirectoryReader(
#     input_files=["./data/10k/lyft_2021.pdf"]
# ).load_data()
# uber_documents = SimpleDirectoryReader(
#     input_files=["./data/10k/uber_2021.pdf"]
# ).load_data()

# lyft_index = VectorStoreIndex.from_documents(lyft_documents)
# uber_index = VectorStoreIndex.from_documents(uber_documents)

# lyft_tool = QueryEngineTool.from_defaults(
#     lyft_index.as_query_engine(),
#     name="lyft_2021",
#     description="Useful for asking questions about Lyft's 2021 10-K filling.",
# )

# uber_tool = QueryEngineTool.from_defaults(
#     uber_index.as_query_engine(),
#     name="uber_2021",
#     description="Useful for asking questions about Uber's 2021 10-K filling.",
# )

### Create Agent

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.agent.multi_hop.planner import MultiHopPlannerAgent

# create the function calling worker for reasoning
worker = FunctionCallingAgentWorker.from_tools([tool], verbose=True)

# wrap the worker in the top-level planner
agent = MultiHopPlannerAgent(worker, tools=[tool], verbose=True)

In [ ]:
agent.create_plan(
    input="Which restaurant chain is headquartered further south, Uncle Maddio's Pizza Joint, or Marco's Pizza?"
)

=== Initial plan ===
## Structured Context
{
    "title": "StructuredContext",
    "description": "Data class for holding data requirements to answer query",
    "type": "object",
    "properties": {
        "uncle_maddios_pizza_joint_headquarters_location": {
            "title": "Uncle Maddios Pizza Joint Headquarters Location",
            "description": "The location of the headquarters of Uncle Maddio's Pizza Joint",
            "type": "string"
        },
        "marcos_pizza_headquarters_location": {
            "title": "Marcos Pizza Headquarters Location",
            "description": "The location of the headquarters of Marco's Pizza",
            "type": "string"
        },
        "restaurant_chain_headquartered_further_south": {
            "title": "Restaurant Chain Headquartered Further South",
            "description": "The restaurant chain that is headquartered further south based on the locations of Uncle Maddio's Pizza Joint and Marco's Pizza",
            "type": "s

'5874152c-b9de-4331-8d3b-0962b2abf1d7'